In [16]:
# import necessary libraries
import warnings
warnings.filterwarnings("ignore")
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
from keras.models import load_model

# create an object of WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# importing the GL Bot corpus file for pre-processing

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open("data.json").read()
data = json.loads(data_file)

In [17]:
# preprocessing the json data
# tokenization
nltk.download('punkt')
nltk.download('wordnet')
for datas in data['data']:
    for pattern in datas['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, datas['tag']))

        # add to our classes list
        if datas['tag'] not in classes:
            classes.append(datas['tag'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [18]:
# Lemmatize and clean words
cleaned_words = []
for w in words:
    if w not in ignore_words:
        cleaned_words.append(lemmatizer.lemmatize(w.lower()))

cleaned_words = sorted(list(set(cleaned_words)))

# Sort classes
classes = sorted(list(set(classes)))

# Create documents by combining patterns and intents
print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(cleaned_words), "unique lemmatized words", cleaned_words)

# Save Python objects using pickle
with open('texts.pkl', 'wb') as file:
    pickle.dump(cleaned_words, file)

with open('labels.pkl', 'wb') as file:
    pickle.dump(classes, file)


249 documents
109 classes ['badmood', 'badmood_followup', 'badmood_followup_response', 'berdamai', 'berdamai_followup1', 'berdamai_followup2', 'berdamai_followup3', 'bete', 'bete_followup', 'bete_followup2_belum_mengerti_solusi', 'bete_followup2_bete', 'bete_followup2_final', 'bete_followup2_masih_bingung_mengatasi_kebosanan', 'bete_followup2_masih_sulit_memaafkan_diri_sendiri', 'bete_followup2_sulit_mengelola_rasa_marah', 'bete_followup2_tidak_tahu_mengatasi_perasaan_jenuh', 'bete_followup2_tidak_yakin_melampiaskan_emosi', 'bete_followup_jenuh', 'bete_followup_marah', 'bete_followup_salah_sendiri', 'capek', 'capek_followup', 'cemas', 'cemas2', 'cemas_followup', 'cemas_followup2', 'cemas_followup3', 'cemas_followup_ancaman_putus', 'cemas_followup_cinta', 'cemas_followup_keluarga', 'cemas_followup_kuliah', 'cemas_followup_uang', 'curhat', 'curhat_end', 'curhat_followup1', 'curhat_followup2', 'dukungan_sosial', 'emosi', 'finansial', 'finansial_followup', 'greeting1', 'greeting2', 'greeti

In [19]:
# Create training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # Loop through words and create bag of words array
    for w in cleaned_words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # Create output row - '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# Shuffle features and convert to NumPy arrays
random.shuffle(training)
train_x = np.array([data[0] for data in training])  # Features
train_y = np.array([data[1] for data in training])  # Labels

print("Training data created")

Training data created


In [20]:
# Create NN model to predict the responses
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fitting the model 
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

# Saving the model
model.save('models.h5')  # Save the model for future use
print("\n")
print("*"*50)
print("\nModel Created Successfully!")


Epoch 1/200
50/50 [==============================] - 1s 1ms/step - loss: 4.6991 - accuracy: 0.0080
Epoch 2/200
50/50 [==============================] - 0s 3ms/step - loss: 4.6631 - accuracy: 0.0241
Epoch 3/200
50/50 [==============================] - 0s 1ms/step - loss: 4.6113 - accuracy: 0.0321
Epoch 4/200
50/50 [==============================] - 0s 1ms/step - loss: 4.5785 - accuracy: 0.0602
Epoch 5/200
50/50 [==============================] - 0s 2ms/step - loss: 4.5244 - accuracy: 0.0602
Epoch 6/200
50/50 [==============================] - 0s 2ms/step - loss: 4.4564 - accuracy: 0.0482
Epoch 7/200
50/50 [==============================] - 0s 1ms/step - loss: 4.4074 - accuracy: 0.0442
Epoch 8/200
50/50 [==============================] - 0s 1ms/step - loss: 4.3499 - accuracy: 0.0602
Epoch 9/200
50/50 [==============================] - 0s 1ms/step - loss: 4.2466 - accuracy: 0.0843
Epoch 10/200
50/50 [==============================] - 0s 1ms/step - loss: 4.1385 - accuracy: 0.1124
Epoch 11/